In [15]:
import pandas as pd
import numpy as np
import sklearn as skl
import nltk as nltk
#import nltk.corenlp as nlp
import seaborn as sbn
import nltk.grammar as grm
import nltk.corpus as corpus


In [16]:
df=pd.read_pickle("/home/rd/Desktop/prj/cleaned_df.pkl")


In [3]:
df

,link,published,title,text,author,label,clean_title,clean_text
0,https://www.nytimes.com/2019/11/20/business/me...,2019-11-21T04:22:14,Media Workers Call Out Pay Gaps in Crowdsource...,With income inequality a focus of the current ...,"[Marc Tracy, Tiffany Hsu]",0,media workers call pay gaps crowdsourced sprea...,income inequality focus current presidential c...
1,https://www.nytimes.com/2019/11/20/business/gm...,2019-11-21T00:07:24,G.M. Sues Rival Over Bribery Scheme as Union S...,Fiat Chrysler “was able to obtain unique advan...,[Neal E. Boudette],0,gm sues rival bribery scheme union scandal exp...,"fiat chrysler ""was able obtain unique advantag..."
2,https://www.nytimes.com/2019/11/20/business/ho...,2019-11-21T01:58:42,U.S. Bill Supporting Hong Kong Rights Heads to...,A bill compelling the United States to support...,"[David Yaffe-Bellany, Alan Rappeport]",0,us bill supporting hong kong rights heads trum...,bill compelling united states support pro demo...
3,https://www.nytimes.com/2019/11/20/business/tr...,2019-11-21T02:08:37,Trump Floats Tariff Exclusions for Apple at Te...,"Jonathan Gold, a spokesman for Americans for F...",[Ana Swanson],0,trump floats tariff exclusions apple texas man...,jonathan gold spokesman americans free trade l...
4,https://www.nytimes.com/2019/11/21/business/de...,2019-11-21T08:00:10,Henry Paulson Sounds Alarm: U.S.-China Relatio...,The United States and China will eventually se...,[Andrew Ross Sorkin],0,henry paulson sounds alarm us china relations ...,united states china eventually settle differen...
5,https://www.nytimes.com/2019/11/20/business/te...,2019-11-20T22:28:32,Tesla’s Winding Road to Berlin,"GRÜNHEIDE, Germany — The visitors from Palo Al...","[Christopher F. Schuetze, Jack Ewing]",0,teslas winding road berlin,grünheide germany visitors palo ao calif shown...
6,https://www.nytimes.com/2019/11/20/technology/...,2019-11-20T22:22:38,Google Hires Firm Known for Anti-Union Efforts,"In August, the company handed down new “commun...","[Noam Scheiber, Daisuke Wakabayashi]",0,google hires firm known anti union efforts,"august company handed new ""community guideline..."
7,https://www.nytimes.com/2019/11/20/business/bu...,2019-11-20T16:19:21,"One Whopper Jr., Hold the Toy","In the climactic scene of “Toy Story 3,” Woody...",[David Yaffe-Bellany],0,one whopper jr hold toy,"climactic scene ""toy story 3"" woody buzz light..."
8,https://www.nytimes.com/2019/11/21/business/jo...,2019-11-21T10:00:20,"Leading AARP, With No Plans to Retire",How has Washington changed during that time?\n...,[David Gelles],0,leading aarp plans retire,washinon changed time? big difference communic...
9,https://www.nytimes.com/2019/11/21/business/de...,2019-11-21T12:07:55,Trump’s Bizarre Apple Factory Visit,Charles Schwab is reportedly in talks to buy T...,[],0,trumps bizarre apple factory visit,charles schwab reportedly talks buy td ameritr...


In [17]:
df.shape
print('----------------')
df.isnull()
print('----------------')
df.isnull().any()

----------------
----------------


link           False
published      False
title          False
text           False
author         False
label          False
clean_title    False
clean_text     False
dtype: bool

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.linear_model import PassiveAggressiveClassifier, LinearRegression
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVR

In [6]:
#x=df['text'] not cleaned , contains many absurd characters
x=df['clean_text']
y=df['label']

In [11]:
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.90)
tfidf_vector=TfidfVectorizer(stop_words='english',max_df=0.70)
tfidf_train=tfidf_vector.fit_transform(train_x)
tfidf_test=tfidf_vector.transform(test_x)

In [12]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,train_y)
pred_y=pac.predict(tfidf_test)
score=accuracy_score(test_y,pred_y)
print('f accuracy:', score)

f accuracy: 0.9881796690307328


In [13]:
#pipeline
from sklearn.pipeline import Pipeline
pipeline=Pipeline([('tfidf',TfidfVectorizer(stop_words='english')),
                  ('svm',LinearSVR())])
pipeline.fit(train_x,train_y)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svm',
                 LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
                           intercept_scaling=1.0, loss='epsilon_

In [14]:
score=pipeline.score(train_x,train_y)
print('accuracy',score)
#the probability of the news may be true or false
#it does not determine whther a news will be completely fake or real !
#just a case of probability and need to know basis for approaching towards any decision making statement

accuracy 0.9999999991278891


In [11]:
pred=pipeline.predict(test_x)
print(classification_report(test_y,pred.round()))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       414
           1       0.00      0.00      0.00         9

    accuracy                           0.98       423
   macro avg       0.49      0.50      0.49       423
weighted avg       0.96      0.98      0.97       423



/home/rd/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
print(confusion_matrix(test_y,pred.round()))

[[414   0]
 [  9   0]]


In [16]:
#web app py and segmentation by making methods and classes